<a href="https://colab.research.google.com/github/YujiSue/Lecture/blob/master/Python%E3%82%B3%E3%83%BC%E3%83%89%E3%82%B5%E3%83%B3%E3%83%97%E3%83%AB%E9%9B%86_%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9%E7%B7%A8_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# リレーショナルデータベース(SQL型)の基本

## 0. [データベースの作成](#01)
## 1. [作成したデータベースを開く](#02)
## 2. [テーブルを作成](#03)
## 3. [テーブルにレコードを挿入](#04)
## 4. [テーブルからレコードを抽出](#05)
## 5. [レコードを編集する](#06)
## 6. [データベースを閉じる](#07)
## Ex 1. [pandasデータフレームをデータベース化](#08)
## Ex 2. [データベースからpandasデータフレームにデータを読み込む](#09)


<a name="01"></a>
## 0. データベースを作る

In [ ]:
#@title データベース名を決めて実行してください
import sqlite3
データベース名 = 'test.db' #@param {type:'string'}
connection = sqlite3.connect(データベース名)
connection.close()

<a name="02"></a>
## 1. データベースを開く

In [ ]:
#@title データベース名を指定して実行してください
import sqlite3
データベース名 = 'test.db' #@param {type:'string'}
connection = sqlite3.connect(データベース名)
cursor = connection.cursor()

<a name="03"></a>
## 2. テーブルをつくる

In [ ]:
#@title テーブルの情報を入力して実行してください
テーブル名 = 'table1' #@param {type:'string'}
カラム数 = 5 #@param {type:'raw'}
#@markdown カラム名をカンマ区切りで入力してください
カラム名 = 'id,name,type,score,photo' #@param {type:'string'}
#@markdown 各カラムのデータ型をカンマ区切りで入力してください
データ型 = 'INTEGER,TEXT,NUMERIC,REAL,BLOB' #@param {type:'string'}
#@markdown 各カラムの制約をカンマ区切りで入力してください
制約 = "PRIMARY KEY,UNIQUE,NOT NULL,DEFAULT 0.0," #@param {type:'string'}

try:
  if カラム数 < 1:
    print('カラム数は１以上に設定してください')
    raise UserWarning('')
  column_names = カラム名.split(',')
  column_types = データ型.split(',')
  column_rules = 制約.split(',')
  if len(column_names) != カラム数 or len(column_types) != カラム数 or len(column_rules) != カラム数:
    print('カラム数とカラム名、データ型、制約の要素数は一致させてください')
    raise UserWarning('')
  sql = 'CREATE TABLE IF NOT EXISTS ' + テーブル名 + '('
  for (cname, ctype, crule) in zip(column_names, column_types, column_rules):
    sql += cname + ' ' + ctype + ' ' + crule + ','
  sql = sql[:-1] + ')'
  print('SQL query :', sql)
  cursor.execute(sql)
  connection.commit()
  print('Completed.')
except:
  print('Failed.')
  pass

<a name="04"></a>
## 3. レコードを挿入する

### １つのレコードを挿入する

In [ ]:
#@title 挿入する情報を入力して実行してください
#@markdown 挿入したいターゲットのテーブル名を入力してください
テーブル名 = 'table1' #@param {type:'string'}
#@markdown 挿入したいターゲットのカラム名をカンマ区切りで入力してください
カラム名 = 'name, type, score' #@param {type:'string'}
#@markdown 挿入するデータをカンマ区切りで入力してください<br>
#@markdown 文字列は半角のシングルクオーテーション(')で囲んでください<br>
データ = "'Item01', 1, 0.5" #@param {type:'string'}

try:
  column_names = カラム名.split(',')
  values = データ.split(',')
  if len(column_names) != len(values):
    print('カラム数とデータ数は一致させてください')
    raise
  if len(column_names) < 1:
    print('１列以上データを入力してください')
    raise
  sql = 'INSERT INTO ' + テーブル名 + '('
  for name in column_names:
    sql += name + ','
  sql = sql[:-1] + ')'
  sql += ' VALUES ('
  for value in values:
    sql += value + ','
  sql = sql[:-1] + ')'
  print('SQL query :', sql)
  cursor.execute(sql)
  connection.commit()
  print('Completed.')
except:
  print('Failed.')
  pass

### ２つ以上のレコードを挿入する

In [ ]:
# 各行をタプルにして、複数行分のデータをリスト化したデータセットを準備します
dataset = [
    ('item02', 1, 0.9),
    ('item03', 2, 0.1),
    ('item04', 3, 0.25),
]

In [ ]:
#@title 挿入する情報を入力して実行してください
#@markdown 挿入したいターゲットのテーブル名を入力してください
テーブル名 = 'table1' #@param {type:'string'}
#@markdown 挿入したいターゲットのカラム名をカンマ区切りで入力してください
カラム名 = 'name, type, score' #@param {type:'string'}
#@markdown 挿入するデータセット名を入力してください<br>
データセット = dataset #@param {type:'raw'}

try:
  column_names = カラム名.split(',')
  if len(column_names) < 1:
    print('１列以上データを入力してください')
    raise
  sql = 'INSERT INTO ' + テーブル名 + '('
  for name in column_names:
    sql += name + ','
  sql = sql[:-1] + ')'
  sql += ' VALUES ('
  for c in range(0, len(column_names)):
    sql += '?,'
  sql = sql[:-1] + ')'
  print('SQL query :', sql)
  cursor.executemany(sql, データセット)
  connection.commit()
  print('Completed.')
except:
  print('Failed.')
  pass

<a name="05"></a>
## 4. レコードを閲覧する

### 1. レコード数をカウントする

In [ ]:
#@title 情報を入力して実行してください
#@markdown カウントしたいターゲットのテーブル名を入力してください
テーブル名 = 'table1' #@param {type:'string'}
try:
  sql = 'select count(*) from ' + テーブル名
  print('SQL query :', sql)
  cursor.execute(sql)
  print('レコード数：', cursor.fetchone()[0])
  print('Completed.')
except:
  print('Failed.')
  pass

### 2. レコードを全て確認する

In [ ]:
#@title 情報を入力して実行してください
#@markdown カウントしたいターゲットのテーブル名を入力してください
テーブル名 = 'table1' #@param {type:'string'}
try:
  sql = 'select * from ' + テーブル名
  print('SQL query :', sql)
  cursor.execute(sql)
  rows = cursor.fetchall()
  ridx = 1
  for row in rows:
    print(ridx, '行目：', row)
    ridx += 1
  print('Completed.')
except:
  print('Failed.')
  pass

### 3. 条件をつけてレコードを検索する


In [ ]:
#@title 情報を入力して実行してください
#@markdown カウントしたいターゲットのテーブル名を入力してください
テーブル名 = 'table1' #@param {type:'string'}
#@markdown 表示したいカラム名をカンマ区切りで入力してください
カラム名 = 'id, name, score' #@param {type:'string'}
#@markdown 表示したいレコードの条件を入力してください
条件 = 'id < 3' #@param {type:'string'}

try:
  sql = 'select ' + カラム名 + ' from ' + テーブル名 + ' WHERE ' + 条件
  print('SQL query :', sql)
  cursor.execute(sql)
  rows = cursor.fetchall()
  ridx = 1
  for row in rows:
    print(ridx, '行目：', row)
    ridx += 1
  print('Completed.')
except:
  print('Failed.')
  pass

<a name="06"></a>
## 5. レコードを編集する

In [ ]:
#@title 編集する情報を入力して実行してください
#@markdown 編集したいターゲットのテーブル名を入力してください
テーブル名 = 'table1' #@param {type:'string'}
#@markdown 編集したいターゲットのカラム名をカンマ区切りで入力してください
カラム名 = 'name, score' #@param {type:'string'}
#@markdown 編集するデータをカンマ区切りで入力してください<br>
#@markdown 文字列は半角のシングルクオーテーション(')で囲んでください<br>
データ = "'Item01_v2', 0.75" #@param {type:'string'}
#@markdown 編集したいデータのIDを指定してください<br>
編集したいID = 1 #@param {type:'number'}

try:
  column_names = カラム名.split(',')
  values = データ.split(',')
  if len(column_names) != len(values):
    print('カラム数とデータ数は一致させてください')
    raise
  if len(column_names) < 1:
    print('１列以上データを入力してください')
    raise
  sql = 'UPDATE ' + テーブル名 + ' SET '
  for name in column_names:
    sql += name + '=?,'
  sql = sql[:-1] + ' '
  sql += 'WHERE ID=' + str(編集したいID)
  print('SQL query :', sql)
  cursor.execute(sql, tuple(values))
  connection.commit()
  print('> 編集後：')
  sql = 'select * from ' + テーブル名
  cursor.execute(sql)
  rows = cursor.fetchall()
  ridx = 1
  for row in rows:
    print(ridx, '行目：', row)
    ridx += 1
  print('Completed.')
except:
  print('Failed.')
  pass

<a name="07"></a>
## 6. データベースを閉じる

In [ ]:
#@title 実行してください
connection.close()

<a name="08"></a>
## Ex 1. Padansとの連携（データベース化） 

In [ ]:
# pandasのデータをデータベース化する
# ここでは日本人に多い苗字ランキング上位10位をデータベース化する<br>
import pandas as pd
import sqlite3

# 苗字リストをデータフレームとして準備する
jpnames = pd.DataFrame([
        [1, '佐藤'],
        [2, '鈴木'],
        [3, '高橋'],
        [4, '田中'],
        [5, '伊藤'],
        [6, '渡辺'],
        [7, '山本'],
        [8, '中村'],
        [9, '小林'],
        [10, '加藤']
])
# name.dbというデータベースを準備する
connection = sqlite3.connect('name.db')
# pandasの関数を利用してデータベースに書き出す
jpnames.to_sql('name',connection,if_exists='replace',index=None)
cursor = connection.cursor()
sql = 'select * from name'
cursor.execute(sql)
rows = cursor.fetchall()
ridx = 1
for row in rows:
  print(ridx, '行目：', row)
  ridx += 1
# データベースを閉じる
connection.close()

<a name="09"></a>
## Ex 2. Padansとの連携（データベースから読み込む） 

In [ ]:
# データベースのデータをpandasにロードする
import pandas as pd
import sqlite3
# 作成したname.dbというデータベースにアクセス
connection = sqlite3.connect('name.db')
# pandasの関数を利用してデータベースから読み出す
data = pd.read_sql('SELECT * FROM name', connection)
print(data)
# データベースを閉じる
connection.close()
# name.csvに書き出す
data.to_csv('name.csv', header=None, index=None)

## 参考文献
1. https://docs.python.org/ja/3/library/sqlite3.html
2. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
3. https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html